In [2]:
#initialization
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math

# importing Qiskit
from qiskit import IBMQ, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

# import basic plot tools
from qiskit.visualization import plot_histogram

# Quantum Phase Estimation Mucho sencillo

1. QFT Dagger applies the inverse of QFT. Using the circuit is the simplest way, but if you want to apply the full one feel free!

2. Conditional Phase gate. We can eithr add one gate with the same phase and repeat it depending on the Qubit (U^n-1). Or just multiply the phase, much simpler.


Find $\theta$ applied by a unitary operator $U$ on its eigenstate $\vert\psi\rangle$ such that

$$U\vert\psi\rangle = e^{2\pi i\theta}\vert\psi\rangle$$

This is done in four main steps.

Apply Superposition to $2^n$ to the first n qubits

We also initialize the target qubits (in this case only one) into an eigenstate $\vert\psi\rangle$ of the unitary operator $U$. 

$$U = \begin{bmatrix}1 & 0\\ 0 & e^{2\pi i\theta}\end{bmatrix}$$

for which an eigenstate is the single-qubit state $\vert1\rangle$. The operator applies a phase

$$U\vert1\rangle = e^{2\pi i\theta}\vert1\rangle$$

In [3]:
from qiskit.circuit.library import QFT
def qft_dagger(circ, qubits, n):
    circ.append(QFT(n).inverse(), qubits)
    
# Add UROTx based on CU1 gates
def apply_crot(circ, control_qubit, target_qubit, theta, exponent):
    circ.cu1(2*math.pi*theta*exponent, control_qubit, target_qubit)

In [4]:
def prepare_circuit(n, theta):
    qpe = QuantumCircuit(n+1,n)
    qpe.x(n)
    qpe.h(range(n))

    for x in range(n):
        exponent = 2**(n-x-1)
        apply_crot(qpe, x, n, theta, exponent)

    qpe.barrier()
    qft_dagger(qpe, range(n), n)
    qpe.barrier()
    for i in range(n):
        qpe.measure(i,i)

    return qpe

In [6]:
# The idea is that we are able to estimate theta (the angle) thanks to the QFT Dagger.
# The more qubits we apply on top, the more accuracy we get. Take a look

import operator

backend = Aer.get_backend('qasm_simulator')
shots = 2048
theta = 0.24125
for qubits in range(2, 15):
    circuit = prepare_circuit(qubits, theta)
    results = execute(circuit, backend=backend, shots=shots).result()
    counts = results.get_counts(circuit)
    
    highest_probability_outcome = max(counts.items(), key=operator.itemgetter(1))[0][::-1]
    measured_theta = int(highest_probability_outcome, 2)/2**qubits
    print("Using %d qubits with theta = %.6f, measured_theta = %.6f." % (qubits, theta, measured_theta))


Using 2 qubits with theta = 0.241250, measured_theta = 0.250000.
Using 3 qubits with theta = 0.241250, measured_theta = 0.250000.
Using 4 qubits with theta = 0.241250, measured_theta = 0.250000.
Using 5 qubits with theta = 0.241250, measured_theta = 0.250000.
Using 6 qubits with theta = 0.241250, measured_theta = 0.234375.
Using 7 qubits with theta = 0.241250, measured_theta = 0.242188.
Using 8 qubits with theta = 0.241250, measured_theta = 0.242188.
Using 9 qubits with theta = 0.241250, measured_theta = 0.242188.
Using 10 qubits with theta = 0.241250, measured_theta = 0.241211.
Using 11 qubits with theta = 0.241250, measured_theta = 0.241211.
Using 12 qubits with theta = 0.241250, measured_theta = 0.241211.
Using 13 qubits with theta = 0.241250, measured_theta = 0.241211.
Using 14 qubits with theta = 0.241250, measured_theta = 0.241272.


In [7]:
## Look at the lovely circuit
circuit.draw('text')

┌───┐                                                             »
 q_0: ┤ H ├─■───────────────────────────────────────────────────────────»
      ├───┤ │                                                           »
 q_1: ┤ H ├─┼───────■───────────────────────────────────────────────────»
      ├───┤ │       │                                                   »
 q_2: ┤ H ├─┼───────┼────────■──────────────────────────────────────────»
      ├───┤ │       │        │                                          »
 q_3: ┤ H ├─┼───────┼────────┼────────■─────────────────────────────────»
      ├───┤ │       │        │        │                                 »
 q_4: ┤ H ├─┼───────┼────────┼────────┼────────■────────────────────────»
      ├───┤ │       │        │        │        │                        »
 q_5: ┤ H ├─┼───────┼────────┼────────┼────────┼───────■────────────────»
      ├───┤ │       │        │        │        │       │                »
 q_6: ┤ H ├─┼───────┼────────┼────────┼────────┼───────┼────────■───────»
      ├───┤ │       │        │        │        │       │        │       »
 q_7: ┤ H ├─┼───────┼────────┼────────┼────────┼───────┼────────┼───────»
      ├───┤ │       │        │        │        │       │        │       »
 q_8: ┤ H ├─┼───────┼────────┼────────┼────────┼───────┼────────┼───────»
      ├───┤ │       │        │        │        │       │        │       »
 q_9: ┤ H ├─┼───────┼────────┼────────┼────────┼───────┼────────┼───────»
      ├───┤ │       │        │        │        │       │        │       »
q_10: ┤ H ├─┼───────┼────────┼────────┼────────┼───────┼────────┼───────»
      ├───┤ │       │        │        │        │       │        │       »
q_11: ┤ H ├─┼───────┼────────┼────────┼────────┼───────┼────────┼───────»
      ├───┤ │       │        │        │        │       │        │       »
q_12: ┤ H ├─┼───────┼────────┼────────┼────────┼───────┼────────┼───────»
      ├───┤ │       │        │        │        │       │        │       »
q_13: ┤ H ├─┼───────┼────────┼────────┼────────┼───────┼────────┼───────»
      ├───┤ │12418  │6208.8  │3104.4  │1552.2  │776.1  │388.05  │194.02 »
q_14: ┤ X ├─■───────■────────■────────■────────■───────■────────■───────»
      └───┘                                                             »
 c_0: ══════════════════════════════════════════════════════════════════»
                                                                        »
 c_1: ══════════════════════════════════════════════════════════════════»
                                                                        »
 c_2: ══════════════════════════════════════════════════════════════════»
                                                                        »
 c_3: ══════════════════════════════════════════════════════════════════»
                                                                        »
 c_4: ══════════════════════════════════════════════════════════════════»
                                                                        »
 c_5: ══════════════════════════════════════════════════════════════════»
                                                                        »
 c_6: ══════════════════════════════════════════════════════════════════»
                                                                        »
 c_7: ══════════════════════════════════════════════════════════════════»
                                                                        »
 c_8: ══════════════════════════════════════════════════════════════════»
                                                                        »
 c_9: ══════════════════════════════════════════════════════════════════»
                                                                        »
c_10: ══════════════════════════════════════════════════════════════════»
                                                                        »
c_11: ══════════════════════════════════════════════════════════════════»
          